In [19]:
import os

import numpy as np
import pandas as pd

from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

### Fresh weights

In [2]:
DIRECTORY = '../weighing/data/destructive/'
file_list = os.listdir(DIRECTORY)
dataset_list = [file for file in file_list if file.endswith('.xlsx')]
dataset_list.sort()

In [3]:
dataset_list

['destructive(0707)_.xlsx',
 'destructive(0708).xlsx',
 'destructive(0709).xlsx',
 'destructive_CT_(0324-0421-0519-0617).xlsx']

In [4]:
FW_df = {}
_ = 0
for FILENAME in dataset_list[-1:]:
    temp_df = pd.read_excel(DIRECTORY + FILENAME, sheet_name=None, skiprows=0)
    for key in temp_df.keys():
        if temp_df[key].shape[0] == 0:
            continue
            
        FW_df[key] = temp_df[key][['Stem FW', 'Leaf FW', 'petiole FW', 'Fruit count', 'Idv fruit FW', 'Plant height']].iloc[:10, :].sum()
    _ += 1 
FW_df = pd.DataFrame.from_dict(FW_df).T

In [5]:
FW_df.index = ['2020-03-24']*3 + ['2020-04-21']*3 + ['2020-05-22']*3 + ['2020-06-17']*4
__ = FW_df

In [6]:
FW_df = {}
_ = 0
for FILENAME in dataset_list[:-1]:
    temp_df = pd.read_excel(DIRECTORY + FILENAME, sheet_name=None, skiprows=0)
    for key in temp_df.keys():
        if temp_df[key].shape[0] == 0:
            continue
            
        FW_df[key] = temp_df[key][['Stem FW', 'Leaf FW', 'petiole FW', 'Fruit count', 'Idv fruit FW', 'Plant height']].iloc[:10, :].sum()
    _ += 1 
FW_df = pd.DataFrame.from_dict(FW_df).T

In [7]:
FW_df = FW_df.filter(like='A2', axis=0)
FW_df.index = ['2020-07-07' for i in range(len(FW_df.index))]

In [8]:
FW_df = pd.concat([__, FW_df], axis=0, sort=False)

In [9]:
weight_df = FW_df.drop(['Plant height', 'Fruit count'], axis=1)

In [10]:
weight_df.to_csv('./results/weight.csv')

In [11]:
weight_df.index

Index(['2020-03-24', '2020-03-24', '2020-03-24', '2020-04-21', '2020-04-21',
       '2020-04-21', '2020-05-22', '2020-05-22', '2020-05-22', '2020-06-17',
       '2020-06-17', '2020-06-17', '2020-06-17', '2020-07-07', '2020-07-07',
       '2020-07-07', '2020-07-07', '2020-07-07', '2020-07-07', '2020-07-07',
       '2020-07-07', '2020-07-07', '2020-07-07', '2020-07-07', '2020-07-07',
       '2020-07-07', '2020-07-07', '2020-07-07', '2020-07-07', '2020-07-07',
       '2020-07-07', '2020-07-07', '2020-07-07'],
      dtype='object')

In [12]:
weight_df = pd.DataFrame(weight_df.sum(axis=1), columns=['FW'])
weight_df['DAT'] = [27]*3 + [55]*3 + [86]*3 + [112]*4 + [132]*20
weight_df = weight_df[['DAT', 'FW']]

In [13]:
def sigmoid(x, L, k, x0, b):
    return L / (b + np.exp(-k*(x-x0)))

In [14]:
x = weight_df['DAT'].values
y = weight_df['FW'].values

In [15]:
popt, pcov = curve_fit(sigmoid, x, y, method='trf')

In [18]:
popt

array([8.21915397e+00, 1.07680612e-01, 3.60741609e+00, 4.40250147e-03])

In [28]:
r2_score(y, sigmoid(x, *popt))

0.7765139814753229

In [16]:
import matplotlib.pyplot as plt

In [17]:
plot_x = [_ for _ in range(0, 140)]

9.19044879 0.092341118 -2.92422326 0.0046724276

In [ ]:
plt.plot(x, y, 'bo', label='data')
plt.plot(plot_x, sigmoid(plot_x, *popt), '-', label='fit')
plt.show()

In [ ]:
weight_df = pd.DataFrame(sigmoid(plot_x, *popt), plot_x, columns=['weight'])
weight_df.to_csv('./results/weight_reg.csv')